In [2]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from keras.models import model_from_json
from keras import optimizers


Using TensorFlow backend.


In [ ]:
data = []
labels = []
total_image = 0

bears = os.listdir('data/bear')
for bear in bears:
    if bear.endswith("jpg") or bear.endswith("png"):
        total_image+=1
        img = cv2.imread("data/bear/" + bear)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(0)
        
cows = os.listdir('data/cows')
for cow in cows:
    if cow.endswith("jpg") or cow.endswith("png"):
        total_image+=1
        img = cv2.imread("data/cows/" + cow)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(1)

deers = os.listdir('data/deer')
for deer in deers:
    if deer.endswith("jpg") or deer.endswith("png"):
        total_image+=1
        img = cv2.imread("data/deer/" + deer)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(2)
        
hogs = os.listdir('data/hog')
for hog in hogs:
    if hog.endswith("jpg") or hog.endswith("png"):
        total_image+=1
        img = cv2.imread("data/hog/" + hog)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(3)
        
turkeys = os.listdir('data/turkey')
for turkey in turkeys:
    if turkey.endswith("jpg") or turkey.endswith("png"):
        total_image+=1
        img = cv2.imread("data/turkey/" + turkey)
        file = Image.fromarray(img,'RGB')
        resize_image = file.resize((50,50))
        data.append(np.array(resize_image))
        labels.append(4)

print("Total Image added : {0}".format(total_image))   


In [ ]:
animals = np.array(data)
labels = np.array(labels)
np.save('animals',animals)
np.save('labels',labels)
s = np.arange(animals.shape[0])
np.random.shuffle(s)
animals = animals[s]
labels = labels[s]
nclasses = len(np.unique(labels))
dlen = len(animals)



In [ ]:
(x_train,x_test)=animals[(int)(0.1*dlen):],animals[:(int)(0.1*dlen)]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255


train_length=len(x_train)
test_length=len(x_test)

(y_train,y_test)=labels[(int)(0.1*dlen):],labels[:(int)(0.1*dlen)]
y_train=keras.utils.to_categorical(y_train,nclasses)
y_test=keras.utils.to_categorical(y_test,nclasses)


model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=96,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(5,activation="softmax"))
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(x_train,y_train,batch_size=50,epochs=1000,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
6657/6657 [==============================] - 11s 2ms/step - loss: 0.9968 - acc: 0.6055
Epoch 2/1000
6657/6657 [==============================] - 6s 827us/step - loss: 0.6183 - acc: 0.7678
Epoch 3/1000
6657/6657 [==============================] - 6s 846us/step - loss: 0.4697 - acc: 0.8328
Epoch 4/1000
6657/6657 [==============================] - 6s 865us/step - loss: 0.4082 - acc: 0.8553
Epoch 5/1000
6657/6657 [==============================] - 6s 905us/step - loss: 0.3418 - acc: 0.8801
Epoch 6/1000
6657/6657 [==============================] - 6s 960us/step - loss: 0.2945 - acc: 0.9025
Epoch 7/1000
6657/6657 [==============================] - 8s 1ms/step - loss: 0.2585 - acc: 0.9130
Epoch 8/1000
6657/6657 [==============================] - 9s 1ms/step - loss: 0.2194 - acc: 0.9261
Epoch 9/1000
6657/6657 [==============================] - 9s 1ms/step - loss: 0.1937 - acc: 0.9371
Epoch 10/1000
6657/6657 [=========================

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test accuracy:', score[1])


In [ ]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((60, 60))
    return np.array(image)

def get_animal_name(label):
    if label==0:
        return "Bear"
    if label==1:
        return "Cow"
    if label==2:
        return "Deer"
    if label==3:
        return "Hog"
    if label==4:
        return "Turkey"
    
    return "Can't Detect"

def predict_animal(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+ animal +" with accuracy =    "+str(acc))    

 

In [ ]:
predict_animal('data/test/13.jpg')


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")